### Codio Activity 9.7: Ridge vs. Sequential Feature Selection

**Expected Time: 60 Minutes**

**Total Points: 40**

This activity focuses on comparing the results of a `Ridge` regression model with that of a `LinearRegression` model built using `SequentialFeatureSelector`.  Both of these approaches seek to limit the complexity of the model.  The `Ridge` estimator applies a penalty that shrinks the coefficients of the model while using the `SequentialFeatureSelector` selects a subset of features to build a model with.  

#### Index

- [Problem 1](#Problem-1)
- [Problem 2](#Problem-2)
- [Problem 3](#Problem-3)
- [Problem 4](#Problem-4)


In [1]:
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn import set_config

set_config(display="diagram")

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

### The Insurance Data

For this example, we return to the insurance data with cubic features.  Below the train and test data is loaded and the train and test sets are determined.  Recall that the target feature has the logarithm applied to it.  

In [2]:
train = pd.read_csv("data/train_cubic.csv")
test = pd.read_csv("data/test_cubic.csv")

In [3]:
X_train, y_train = train.drop("target_log", axis=1), train["target_log"]
X_test, y_test = test.drop("target_log", axis=1), test["target_log"]

### Problem 1

#### Feature Selection Pipeline

**10 Points**

To begin, use the pipeline below to construct a grid search over the `n_features_to_select` parameter of the `SequentialFeatureSelector` transformer.  Consider 2, 3, 4, and 5 features in your search.  Create the dictionary to be used in the search as `param_dict`.  

Assign your grid to `selector_grid`, fit on the training data, and determine the mean squared error on the train and test set.  Assign the errors as floats to `selector_train_mse` and `selector_test_mse` respectively.

In [4]:
selector_pipe = Pipeline(
    [
        ("selector", SequentialFeatureSelector(LinearRegression())),
        ("model", LinearRegression()),
    ]
)
selector_pipe

Pipeline(steps=[('selector',
                 SequentialFeatureSelector(estimator=LinearRegression())),
                ('model', LinearRegression())])

In [5]:
### GRADED

param_dict = {"selector__n_features_to_select": [2, 3, 4, 5]}
selector_grid = GridSearchCV(estimator=selector_pipe, param_grid=param_dict).fit(
    X_train, y_train
)
selector_train_mse = mean_squared_error(selector_grid.predict(X_train), y_train)
selector_test_mse = mean_squared_error(selector_grid.predict(X_test), y_test)

# ANSWER CHECK
print(f"Train MSE: {selector_train_mse}")
print(f"Test MSE: {selector_test_mse}")

Train MSE: 0.6031734290034885
Test MSE: 0.5655875591380699


### Problem 2

#### Ridge Grid

**10 Points**

Now, construct a `Pipeline` that contains two steps -- `scaler` and `ridge` that first standard scales the data and then build a ridge regression model.  Assign your pipeline as `ridge_pipe`.  Use this to execute the grid search over the `alpha` hyperparameter of the `Ridge` estimator using the training data. Determine the mean squared error on the train and test data. 

Assign the errors as floats to `ridge_train_mse` and `ridge_test_mse` respectively.


Hint: 

Define a parameter dictionary named `ridge_param_dict` for the grid search. For this, use `np.logspace(0, 10, 50)` to create a range of alpha values `ridge__alpha`. This function generates values evenly spaced in logarithmic scale from 1 to 10^10. The parameter dictionary is specified as follows: `ridge_param_dict = {'ridge__alpha': np.logspace(0, 10, 50)}`.

After defining `ridge_param_dict`, use it to execute the grid search over the alpha hyperparameter using the training data. Determine the mean squared error on the train and test data and assign the errors as floats to `ridge_train_mse` and `ridge_test_mse` respectively.

In [6]:
### GRADED

ridge_param_dict = {"ridge__alpha": np.logspace(0, 10, 50)}
ridge_pipe = Pipeline([("scaler", StandardScaler()), ("ridge", Ridge())])
ridge_grid = GridSearchCV(estimator=ridge_pipe, param_grid=ridge_param_dict).fit(
    X_train, y_train
)
ridge_train_mse = mean_squared_error(ridge_grid.predict(X_train), y_train)
ridge_test_mse = mean_squared_error(ridge_grid.predict(X_test), y_test)

# ANSWER CHECK
print(f"Train MSE: {ridge_train_mse}")
print(f"Test MSE: {ridge_test_mse}")
ridge_pipe

Train MSE: 0.5870277750390883
Test MSE: 0.5532169282339893


Pipeline(steps=[('scaler', StandardScaler()), ('ridge', Ridge())])

### Problem 3

#### Examining the "best" model

**10 Points**

Your results should suggest that the model using the sequential feature selector and `LinearRegression` estimator.  This was fit with the object `selector_grid`.  One question we may have is what was the optimal number of features selected and what were they?  

Use the `selector_grid` to extract both the feature names and their associated coefficients.  This will involve:

- `.best_estimator_`: extract the best estimator/selector pair from your grid search
- `.named_steps['selector']`: extract the selector from the pipeline
- `.named_steps['model']`: extract the model from the pipeline
- `.get_support()`: extract best features from selector.  This returns booleans as to whether feature was selected, we can use this to slice our train data.  

```python
X_train.columns[best_selector.get_support()]
```

- `.coef_`: coefficients from best model

In [7]:
### GRADED

best_estimator = selector_grid.best_estimator_
best_selector = best_estimator.named_steps["selector"]
best_model = best_estimator.named_steps["model"]
feature_names = best_selector.get_feature_names_out()
coefs = best_model.coef_

# Answer check
print(best_estimator)
print(f"Features from best selector: {feature_names}.")
print("Coefficient values: ")
print("===================")
pd.DataFrame([coefs.T], columns=feature_names, index=["model"])

Pipeline(steps=[('selector',
                 SequentialFeatureSelector(estimator=LinearRegression(),
                                           n_features_to_select=2)),
                ('model', LinearRegression())])
Features from best selector: ['age' 'bmi children'].
Coefficient values: 


,age,bmi children
model,0.032852,0.00368


### Problem 4

#### Comparing observations 

**10 Points**

According to your model, predict the billed costs for person 1 and person 2 below:

- **Person 1**: Age = 30, bmi = 40, children = 0
- **Person 2**: Age = 45, bmi = 50, children = 2

Use the information from **Problem 3** and the model coefficients to make these predictions.

Note that you will want to transform your predictions.  From your model the predictions are in terms of the logarithm of cost.  To transform the logarithm to the actual value, use `np.exp` -- the inverse of a logarithm. Assign your predictions as floats to `person1` and `person2` below.  Your solution will be checked to two decimal point accuracy. 

In [11]:
### GRADED
ages = [30, 45]
bmis = [40, 50]
childrens = [0, 2]
person1 = float(np.exp(best_model.predict([[ages[0], bmis[0] * childrens[0]]]))[0])
person2 = float(np.exp(best_model.predict([[ages[1], bmis[1] * childrens[1]]]))[0])
# display(np.exp(best_model.predict(pd.DataFrame({"age":ages, "bmi children":np.array(bmis)*np.array(childrens)}))))

# Answer check
print(f"The difference between Person 1 and Person 2 is {person2 - person1: .2f}")

The difference between Person 1 and Person 2 is  8052.04


The models here could be revisited and more encoding of features and different polynomial terms can be incorporated.  More important is understanding how to construct the pipelines and interrogate the resulting models to understand what they say about your data.  Does having a higher body mass matter if one does not have children?  Does this seem reasonable?